In [7]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)


<module 'model.modelAI' from 'D:\\proyectos\\OSMOSYS_2023\\IntegracionAI2023\\AI_OsmosysIntegrationScripts2023\\model\\modelAI.py'>

In [9]:
indicatorCodeAI = 'PRPI1_02'
month = 'ENERO'
month_number = '2023-01'
year = 2023
test = False
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')

--------------------------------------PRPI1_02------------------------------------------------------


In [3]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf.info)
indicatorIdsOsmosys = subformDf.osmosys_indicator_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicador_ai_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: cmdhbwlldnik44oiga
parentFormIdAI: crphcl4ldnik44oig8
indicatorIdAI: c1oy9mvlcza9pfv51
indicatorIdsOsmosys: [198 200]


In [4]:
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))

reported data: 6


In [5]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [10]:
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    PRPI1_02_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRPI1_02_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRPI1_02_RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                   cantonCode=cantonCode)
    #print(dfDiversidad)
    PRPI1_02_RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    PRPI1_02_RM_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[0].value_a
    PRPI1_02_CUAL = commentary

    subform = model.modelAI.SubFormPRPI1_02(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        poblacion_meta2='Refugiados y migrantes',
        PRPI1_02_RM_NA=PRPI1_02_RM_NA,
        PRPI1_02_RM_NN=PRPI1_02_RM_NN,
        PRPI1_02_RM_OTR=PRPI1_02_RM_OTR,
        PRPI1_02_RM_LGBT=PRPI1_02_RM_LGBT,
        PRPI1_02_RM_DS=PRPI1_02_RM_DS,
        PRPI1_02_CUAL=PRPI1_02_CUAL

    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 5


In [11]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))


 creado respaldo: ENERO\data_PRPI1_02.json


In [12]:
if (test):
    print(
        '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

se envia a AI
--------------------------------------PRPI1_02------------------------------------------------------
